In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## object detection 파일 분리
 - train/test split

In [ ]:
import os
import glob
import shutil
import random

# 데이터 분할 및 이동
def split_and_move_data(data_path, split_ratio=3/4):   # train:test = 6:2로 분리 (valid 2 분리되어있음)
    htp_list = ['house','person','tree']

    for htp in htp_list:
        # 해당하는 image 파일 리스트 추출
        image_files = glob.glob(os.path.join(data_path, htp, 'train','images', '*.jpg'))

        # 데이터 순서를 랜덤하게 섞음
        random.shuffle(image_files)

        # 전체 데이터셋 중 split_ratio 만큼 train 길이 계산
        num_train = int(len(image_files) * split_ratio)

        # print(htp, num_train, len(image_files) - num_train)

        # copy 대상 폴더 경로 지정
        new_data_path = '/content/drive/MyDrive/HTP/detection/'

        # train/test 폴더 지정
        for i, image_file in enumerate(image_files):
            if i < num_train:
                split = 'train'
            else:
                split = 'test'

            # 이미지 데이터 이동
            dest_image_path = os.path.join(new_data_path, htp, split ,'images', os.path.basename(image_file))
            shutil.copy(image_file, dest_image_path)

            # 레이블 데이터 파일명을 맞춰서 이동
            label_path = os.path.splitext(image_file)[0].replace('images','labels')
            label_file = label_path + '.txt'
            dest_label_path = os.path.join(new_data_path, htp, split, 'labels', os.path.basename(label_file))
            shutil.copy(label_file, dest_label_path)

if __name__ == '__main__':
    data_path = '/content/drive/MyDrive/DBI/detection/'

    split_and_move_data(data_path)

 - valid copy

In [ ]:
htp_list = ['house','person','tree']

# 기존 경로
data_path = '/content/drive/MyDrive/DBI/detection/'
# 이동할 경로
new_path = '/content/drive/MyDrive/HTP/detection/'

# 이미지/레이블 데이터 이동
for htp in htp_list:
    image_path = data_path + htp + '/valid/images'
    label_path = data_path + htp + '/valid/labels'
    dest_image_path = new_path + htp + '/valid/images'
    dest_label_path = new_path + htp + '/valid/labels'

    print(htp)
    shutil.copytree(image_path, dest_image_path)
    shutil.copytree(label_path, dest_label_path)

### 나눈 파일들을 drive mount에 저장하고, 불러오기

In [ ]:
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/detection.zip') as target_file:
    target_file.extractall('/content/')

## model 구축
 - yaml파일 test, train, valid image 경로 수정해줘야 한다!

In [ ]:
!pip install ultralytics

- 최적 파라미터 구하기

In [ ]:
from sklearn.model_selection import ParameterGrid
from ultralytics import YOLO

learning_rates = [0.001, 0.01, 0.01, 0.1]
batch_sizes = [16, 32, 64]

search_space = {'learning_rate': learning_rates,
                'batch_size': batch_sizes,
}

parameter_grid = ParameterGrid(search_space)
# 최적의 파라미터 (learning_rate, batch_size)를 찾는 함수
def get_parameter(data_path):
  for parameters in parameter_grid:
    learning_rate = parameters['learning_rate']
    batch_size = parameters['batch_size']
    model = YOLO('yolov8n.pt')
    model.train(data=data_path, epochs=100, patience=10, batch = batch_size, imgsz=640, lr0=learning_rate)


- house 모델 구축

In [ ]:
house_data_path = '/content/detection/house/house.yaml'
get_parameter(house_data_path)

In [ ]:
house_model = YOLO('yolov8n-seg.pt')
house_model.train(data='/content/segmentation/house/data.yaml', epochs=500, patience=50, batch = 16, imgsz=640, lr0=0.01)

Ultralytics YOLOv8.0.195 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/segmentation/house/data.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, k

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79e8f62e21d0>
fitness: 1.8322667220328586
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([     1.8209])
names: {0: 'house'}
plot: True
results_dict: {'metrics/precision(B)': 0.9059173694710949, 'metrics/recall(B)': 0.9362745098039216, 'metrics/mAP50(B)': 0.960972484713441, 'metrics/mAP50-95(B)': 0.8965216565269353, 'metrics/precision(M)': 0.9154034152247189, 'metrics/recall(M)': 0.946078431372549, 'metrics/mAP50(M)': 0.9733719318842677, 'metrics/mAP50-95(M)': 0.9244030994431622, 'fitness': 1.8322667220328586}
save_dir: PosixPath('runs/segment/train5')
seg: ultralytics.utils.metrics.Metric object
speed: {'preprocess': 5.33988614561696

In [ ]:
# 결과
house_results = house_model.predict(source='/content/detection/house/test/images/', save_txt=True)

 - person 모델 구축

In [ ]:
person_data_path = '/content/detection/person/person.yaml'
get_parameter(person_data_path)

In [ ]:
from ultralytics import YOLO
person_model = YOLO('yolov8n.pt')
person_model.train(data='/content/detection/person/person.yaml', epochs=500, patience=50, batch = 16, imgsz=640, lr0=0.01)

100%|██████████| 6.23M/6.23M [00:00<00:00, 78.8MB/s]
Ultralytics YOLOv8.0.195 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/detection/person/person.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f730442290>
fitness: 0.6772028939429667
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.93516,      0.5553,     0.49855,     0.61276,     0.66196])
names: {0: 'person', 1: 'eye', 2: 'mouth', 3: 'arm', 4: 'leg'}
plot: True
results_dict: {'metrics/precision(B)': 0.8951324087344815, 'metrics/recall(B)': 0.8416267990165757, 'metrics/mAP50(B)': 0.8973336028541963, 'metrics/mAP50-95(B)': 0.6527439262861634, 'fitness': 0.6772028939429667}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 3.1556200981140137, 'inference': 3.3581912517547607, 'loss': 0.0007200241088867188, 'postprocess': 3.386664390563965}

In [ ]:
# 결과
person_results = person_model.predict(source='/content/detection/person/test/images/', save_txt=True)

 - tree 모델 구축

In [ ]:
tree_data_path = '/content/detection/tree/tree.yaml'
get_parameter(tree_data_path)

In [ ]:
tree_model = YOLO('yolov8n.pt')
tree_model.train(data='/content/detection/tree/tree.yaml', epochs=500, patience=50, batch = 16, imgsz=640, lr0=0.01)

Ultralytics YOLOv8.0.195 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/detection/tree/tree.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fals

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f72ffa09a0>
fitness: 0.5830321705980558
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([     0.8998,     0.45638,     0.80148,     0.30263,     0.59528,     0.32362])
names: {0: 'tree', 1: 'gnarl', 2: 'crown', 3: 'branch', 4: 'root', 5: 'fruits'}
plot: True
results_dict: {'metrics/precision(B)': 0.8179527786205346, 'metrics/recall(B)': 0.7028205458897349, 'metrics/mAP50(B)': 0.7615334266199828, 'metrics/mAP50-95(B)': 0.5631986977067306, 'fitness': 0.5830321705980558}
save_dir: PosixPath('runs/detect/train2')
speed: {'preprocess': 1.6297913612203394, 'inference': 4.139877380208767, 'loss': 0.000651846540735123, 'postprocess': 3.2370103166458457}

In [ ]:
# 결과
tree_results = tree_model.predict(source='/content/detection/tree/test/images/', save_txt=True)